Seguindo o guia de API do youtube para python, fazemos o pip install de algumas bibliotecas específicas para APIs google. Referência abaixo:

https://developers.google.com/youtube/v3/quickstart/python?hl=pt-br

In [ ]:
pip install --upgrade google-api-python-client

In [ ]:
pip install openai

Importação de bibliotecas

In [54]:
import pandas as pd
import json
import requests
import googleapiclient.discovery 
import os
import openai

Em seguida no painel biblioteca https://console.developers.google.com/apis/library?hl=pt-br encontre API YouTube Data v3 e ative a API no seu projeto.

Em credenciais criei uma chave de API que não necessita de autorização do usuário para usar dados públicos e em suas configurações restringi o uso para youtube v3.

no própio tutorial do site eles ensinam a fazer essa função built para consumir os dados da API. Lembrando que compartilhei o código sem as chaves.

Extração de dados da API do yutube

Comecei construindo uma função que extrai os Ids de todos os vídeos de uma playlist (identificada também pelo seu id), para este projeto utilizei a playlist "Data & Analytcs" da DIO ❤️ playlist_Id = PLPFhNx6dMDJq1YsZmlO6BFuFBayUh84IA . (Coloque sua chave de API no local indicado)

In [55]:
key = 'Sua chave aqui'

def get_video_id(playlist_Id):
        
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey = key)  
    nextPage_token = None
    videos_ids = []

    while True:
        request = youtube.playlistItems().list(part = "snippet", pageToken = nextPage_token, playlistId = playlist_Id)
        response = request.execute()
        for item in response["items"]:
            video_id = item["snippet"]["resourceId"]["videoId"]
            videos_ids.append(video_id)
        nextPage_token = response.get('nextPageToken')
        if nextPage_token is None:
            break
    
    return videos_ids 
   

Extração parte 2

Em seguida, fiz uma função para extrair todos os comentários de cada vídeo por video_id (Coloque sua chave de API no local indicado)

In [56]:
key = 'Sua chave aqui'

def get_comment(video_id):
    
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey = key)  
    nextPage_token = None
    comments = []

    while True:
        request = youtube.commentThreads().list( part = 'snippet, replies', pageToken = nextPage_token, videoId = video_id, textFormat="plainText")
        response = request.execute()
        for item in response["items"]:
            snippet = item["snippet"]
            comment = snippet["topLevelComment"]["snippet"]["textDisplay"]
            comments.append(comment)
        nextPage_token = response.get('nextPageToken')
        if nextPage_token is None:
            break

    return comments

Transformação de dados

Integração com chat GPT com a biblioteca openai
O compartilhamento do arquivo foi feito sem a chave da openai
A função generate_ai_analysis(video_id) classifica e contabiliza os comentários de cada vídeo como positivo, negativo ou neutro através da IA
(Coloque sua chave de API no local indicado)

In [57]:
openai_api_key = 'Sua chave aqui'

openai.api_key = openai_api_key

def generate_ai_analysis(video_id):
    comments = get_comment(video_id)
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
         messages=[
            {"role": "system", "content": "Você é um analista de sentimentos em comentários."},
            {"role": "user", "content": f"contabilize quantas mensagens em {comments} são positivas, quantas são neutras e quantas são negativas, como resposta mostre a soma agrupada em positivas, negativas ou neutras(máximo 30 caracteres)"}
        ]
    )
    responseChatGPT = completion.choices[0].message.content.strip('\"')
    return responseChatGPT

Carregamento 

Esta função retorna um comentário para o vídeo.

Para esta função, foi necessário criar uma credencial com o Google OAuth 2.0. Precisei assistir a alguns tutoriais e criar um server, mas saiu. O arquivo do server está anexado no projeto. (Crie uma chave OAuth 2.0, baixe o arquivo json e coloque na mesma pasta desse projeto)

In [ ]:
pip install google-auth-oauthlib

Autenticação:

In [58]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"])

flow.run_local_server(port = 8000, prompt = "consent", authorization_prompt_message = "")

credentials = flow.credentials

Carregamento
Essa é a função de update propriamente dita, ela recebe un texto e um video id e retorna mensagem ao video correspondente.

In [62]:
def update_comment_analysis(video_id, text):
  youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials = credentials)
  request = youtube.commentThreads().insert(part="snippet",
        body={
          "snippet": {
            "videoId": video_id,
            "topLevelComment": {
              "snippet": {
                "textOriginal": text
              }
            }
          }
        }
      )
  response = request.execute()
  return response

No próximo bloco eu fiz um pequeno script unindo todas as funções que retorna automaticamente a análise dos comentários de todos os vídeos de uma playlist. Como eu tenho uma versão gratuita do chat GPT, tenho uma restrição de requests por minuto, que contornei adicionando a função time.sleep da bibliteca time(nativa do python). Aparentemente, criei meu primeiro robô! Muito feliz por esse projeto. Agradeço a todos da DIO, principalmente ao grande professor Venilton Falvojr 💕

In [64]:
import time
playlist_Id = input("Digite o Id da playList a ser analisada sem aspas")

videos_id = get_video_id(playlist_Id) 
for video_id in videos_id:
    analysis = generate_ai_analysis(video_id)
    text = f"Até o momento a análise de comentários desse vídeo resulta em: \n\nClassificação das mensagens\n{analysis} \n\nMensagem automática gerada por chat GPT para desafio do bootcamp DIO/Santander\n#Makingthechange\n@Santander @DIO"
    update_comment_analysis(video_id, text)
    time.sleep(30)